In [1]:
import requests
from bs4 import BeautifulSoup


search_query = input("Enter the product to search: ")


url = f"https://www.amazon.in/s?k={search_query}&ref=nb_sb_noss_2"


response = requests.get(url).content


soup = BeautifulSoup(response, 'html.parser')


products = soup.find_all('div', {'data-component-type': 's-search-result'})


for product in products:
    
    name = product.find('span', {'class': 'a-size-medium'}).text.strip()
    
    
    try:
        price = product.find('span', {'class': 'a-price-whole'}).text.strip()
        currency = product.find('span', {'class': 'a-price-symbol'}).text.strip()
        price = currency + price
    except AttributeError:
        price = "Not available"
    
    
    try:
        rating = product.find('span', {'class': 'a-icon-alt'}).text.strip()
    except AttributeError:
        rating = "Not available"
    
    
    print(f"Product: {name}")
    print(f"Price: {price}")
    print(f"Rating: {rating}")
    print("="*50)


Enter the product to search: guitars


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


search_query = input("Enter the product to search: ")


product_data = []


max_pages = 3


page_num = 1

while page_num <= max_pages:
    
    url = f"https://www.amazon.in/s?k={search_query}&page={page_num}"
    
    
    response = requests.get(url).content
    
    
    soup = BeautifulSoup(response, 'html.parser')
    
    
    products = soup.find_all('div', {'data-component-type': 's-search-result'})
    
    
    for product in products:
        
        try:
            name = product.find('span', {'class': 'a-size-medium'}).text.strip()
            url = "https://www.amazon.in" + product.find('a', {'class': 'a-link-normal'})['href']
        except AttributeError:
            continue
        
        
        try:
            brand = product.find('span', {'class': 'a-size-base-plus'}).text.strip()
        except AttributeError:
            brand = "-"
        
        
        try:
            price = product.find('span', {'class': 'a-price-whole'}).text.strip()
            currency = product.find('span', {'class': 'a-price-symbol'}).text.strip()
            price = currency + price
        except AttributeError:
            price = "-"
        
        
        try:
            return_policy = product.find('div', {'class': 'a-column a-span2'}).text.strip()
        except AttributeError:
            return_policy = "-"
        
        
        try:
            delivery = product.find('div', {'class': 'a-column a-span3'}).text.strip()
            delivery = re.sub(r'\n', '', delivery)
        except AttributeError:
            delivery = "-"
        
        
        try:
            availability = product.find('div', {'class': 'a-column a-span7'}).text.strip()
        except AttributeError:
            availability = "-"
        
        
        product_data.append([brand, name, price, return_policy, delivery, availability, url])
    
    
    next_button = soup.find('span', {'class': 's-pagination-item s-pagination-next s-pagination-button'})
    if next_button is None:
        break
    
    
    page_num += 1


df = pd.DataFrame(product_data, columns=['Brand', 'Name', 'Price', 'Return/Exchange', 'Expected Delivery', 'Availability', 'Product URL'])


df.to_csv(f"{search_query}.csv", index=False)


print(f"Data saved to {search_query}.csv")


Enter the product to search: guitar
Data saved to guitar.csv


In [4]:
import os
import requests
from bs4 import BeautifulSoup


keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']


if not os.path.exists('images'):
    os.makedirs('images')


for keyword in keywords:
    
    url = f"https://www.google.com/search?q={keyword}&tbm=isch"
    
    
    response = requests.get(url)
    
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    
    images = soup.find_all('img')
    
   
    for i, image in enumerate(images[:10]):
        image_url = image['src']
        image_data = requests.get(image_url).content
        with open(f"images/{keyword}_{i}.jpg", "wb") as f:
            f.write(image_data)


MissingSchema: Invalid URL '/images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif': No scheme supplied. Perhaps you meant http:///images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif?

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


search_url = 'https://www.flipkart.com/search'
params = {'q': input('Enter a smartphone name: '), 'otracker': 'search', 'otracker1': 'search', 'marketplace': 'FLIPKART', 'as-show': 'on', 'sort': ''}
response = requests.get(search_url, params=params)


soup = BeautifulSoup(response.content, 'html.parser')


products = soup.find_all('div', {'class': '_2kHMtA'})


brand_list = []
name_list = []
color_list = []
ram_list = []
rom_list = []
primary_camera_list = []
secondary_camera_list = []
display_size_list = []
battery_capacity_list = []
price_list = []
url_list = []


for product in products:
   
    title = product.find('div', {'class': '_4rR01T'})
    brand_name, smartphone_name = title.text.split(' ', 1)
    
    
    color = product.find('div', {'class': '_3IrTcX'})
    color_text = color.text if color else '-'
    
    
    specs = product.find('ul', {'class': '_1xgFaf'})
    spec_list = specs.text.split('|')
    ram = '-'
    rom = '-'
    for spec in spec_list:
        if 'RAM' in spec:
            ram = spec.strip()
        elif 'ROM' in spec:
            rom = spec.strip()
    
    
    camera = product.find('ul', {'class': '_1xgFaf'}, text='Camera')
    camera_list = camera.parent.text.split('|')
    primary_camera = '-'
    secondary_camera = '-'
    for c in camera_list:
        if 'MP' in c:
            if primary_camera == '-':
                primary_camera = c.strip()
            else:
                secondary_camera = c.strip()
    
    
    display = product.find('ul', {'class': '_1xgFaf'}, text='Display')
    display_size = display.parent.text.split('|')[0].strip() if display else '-'
    
    
    battery = product.find('ul', {'class': '_1xgFaf'}, text='Battery')
    battery_capacity = battery.parent.text.split('|')[0].strip() if battery else '-'
    
    
    price = product.find('div', {'class': '_30jeq3 _1_WHN1'})
    url = product.find('a', {'class': '_1fQZEK'})['href']
    
    
    brand_list.append(brand_name)
    name_list.append(smartphone_name)
    color_list.append(color_text)
    ram_list.append(ram)
    rom_list.append(rom)
    primary_camera_list.append(primary_camera)
    secondary_camera_list.append(secondary_camera)
    display_size_list.append(display_size)
    battery_capacity_list.append(battery_capacity)
    price_list.append(price.text if price else '-')
    url_list


In [1]:
import requests


url = 'https://maps.googleapis.com/maps/api/geocode/json'


city_name = input('Enter the city name: ')


params = {'address': city_name,'key': 'YOUR_API_KEY_HERE'}

response = requests.get(url, params=params)
data = response.json()


if data['status'] == 'OK':
    
    lat = data['results'][0]['geometry']['location']['lat']
    lng = data['results'][0]['geometry']['location']['lng']
    
    
    print(f'The geospatial coordinates of {city_name} are: ({lat}, {lng})')
else:
    print('Error: Could not find the geospatial coordinates.')


Enter the city name: Lagos
Error: Could not find the geospatial coordinates.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://trak.in/india-startup-funding-investment-2015/'


response = requests.get(url)


soup = BeautifulSoup(response.content, 'html.parser')


table = soup.find_all('table')[6]


headers = [header.text.strip() for header in table.find_all('th')]


rows = []
for row in table.find_all('tr')[1:]:
    rows.append([data.text.strip() for data in row.find_all('td')])


q2_rows = []
for row in rows:
    if row and row[0].startswith('Q2') and '2021' in row[0]:
        q2_rows.append(row)


df = pd.DataFrame(q2_rows, columns=headers)


df.to_csv('funding_deals_q2_2021.csv', index=False)


print(df)


Empty DataFrame
Columns: [Sr. No., Date (dd/mm/yyyy), Startup Name, Industry / Vertical, Sub-Vertical, City / Location, Investors' Name, Investment Type, Amount (In USD)]
Index: []


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

laptop_names = [laptop.text for laptop in soup.find_all("div", {"class": "TopNumbeHeading"})]

specifications = [spec.text.strip().replace('\n\n\n\n\n\n', '\n') for spec in soup.find_all("div", {"class": "Spcs-lft"})]

spec_values = []
for spec in soup.find_all("div", {"class": "Spcs-rght"}):  
    value = spec.text.strip().replace('\n\n\n\n\n\n\n\n\n\n\n', ',').replace('\n\n\n\n\n\n\n\n\n\n', ',').replace('\n\n\n\n\n\n', ',').replace('\n\n\n', ',').replace('\n', ',')  
    spec_values.append(value)

laptops = []
for i in range(len(laptop_names)):  
    laptop = {}   
    laptop["Name"] = laptop_names[i]  
    specs = specifications[i].split("\n")    
    values = spec_values[i].split(",")   
    for j in range(len(specs)):     
        laptop[specs[j]] = values[j]   
        laptops.append(laptop)

df = pd.DataFrame(laptops)

df.to_csv("best_gaming_laptops.csv", index=False)

SyntaxError: invalid non-printable character U+00A0 (4228678970.py, line 16)

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "https://www.forbes.com/billionaires/"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')


table = soup.find('table', {'class': 'table'})


ranks = []
names = []
net_worths = []
ages = []
citizenships = []
sources = []
industries = []


for row in table.find_all('tr')[1:]:
    
    rank = row.find_all('td')[0].text.strip()
    name = row.find_all('td')[1].text.strip()
    net_worth = row.find_all('td')[2].text.strip()
    age = row.find_all('td')[3].text.strip()
    citizenship = row.find_all('td')[4].text.strip()
    source = row.find_all('td')[5].text.strip()
    industry = row.find_all('td')[6].text.strip()

    
    ranks.append(rank)
    names.append(name)
    net_worths.append(net_worth)
    ages.append(age)
    citizenships.append(citizenship)
    sources.append(source)
    industries.append(industry)


df = pd.DataFrame({'Rank': ranks,'Name': names,'Net Worth': net_worths,'Age': ages,'Citizenship': citizenships,'Source': sources,'Industry': industries})


df.to_csv('billionaires.csv', index=False)


AttributeError: 'NoneType' object has no attribute 'find_all'

In [8]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 178.9/178.9 kB 269.8 kB/s eta 0:00:00
     ---------------------------------------- 11.3/11.3 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 96.9/96.9 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 120.3/120.3 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 422.5/422.5 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 223.6/223.6 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
import os
import google.auth
from googleapiclient.discovery import build

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_key = "<your api key>"


video_id = "<your video ID>"


youtube = build("youtube", "v3", developerKey=api_key)


next_page_token = None
comments = []
num_comments = 0


while num_comments < 500:
    
    response = youtube.commentThreads().list(part="snippet",videoId=video_id,pageToken=next_page_token,maxResults=100 ).execute()

    
    for item in response["items"]:
        comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        upvotes = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
        posted_at = item["snippet"]["topLevelComment"]["snippet"]["publishedAt"]
        comments.append({"comment": comment, "upvotes": upvotes, "posted_at": posted_at})
        num_comments += 1

    
    if "nextPageToken" in response:
        next_page_token = response["nextPageToken"]
    else:
        break


for comment in comments:
    print(comment)


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=%3Cyour+video+ID%3E&maxResults=100&key=%3Cyour+api+key%3E&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = 'https://www.hostelworld.com/findabed.php/ChosenCity.London/ChosenCountry.England'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
hostels = soup.find_all('div', class_='property-card')
data = []
for hostel in hostels:   
    name = hostel.find('h2', class_='title').text.strip()    
    distance = hostel.find('span', class_='description').text.strip()   
    rating = hostel.find('div', class_='score orange').text.strip()    
    total_reviews = hostel.find('div', class_='reviews').text.strip()    
    overall_reviews = hostel.find('div', class_='keyword').text.strip()    
    prices = hostel.find_all('div', class_='price-col')    
    privates_from_price = prices[0].find('a').text.strip()   
    dorms_from_price = prices[1].find('a').text.strip()    
    facilities = [fac.text.strip() 
                  for fac in hostel.find_all('div', class_='facilities')]   
    description = hostel.find('div', class_='description').text.strip()
    data.append({        'Name': name,        'Distance from City Centre': distance,        'Rating': rating,        'Total Reviews': total_reviews,        'Overall Reviews': overall_reviews,        'Privates from Price': privates_from_price,        'Dorms from Price': dorms_from_price,        'Facilities': facilities,        'Property Description': description})
df = pd.DataFrame(data)df.to_csv('hostels_london.csv', index=False)print('Scraping complete.')

SyntaxError: invalid non-printable character U+00A0 (1301208812.py, line 9)